In [2]:
%matplotlib widget
import os
import sys
path = os.path.dirname(os.path.abspath(""))+"/"
sys.path.append(path)
print(path)

/Users/Paul/Harvard/Code/connectomics/


In [3]:
from skimage import io
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import glob
from CellAnalysis.eval import Eval
from CellAnalysis.eval import Benchmarker
from matplotlib.gridspec import GridSpec
import seaborn as sns

In [4]:
def load_2d_masks(root_dir, mask_list=None, keyword='mask', ignore_keyword=False, dtype=np.uint16):
    if mask_list is None:
        mask_list = []
    for name in sorted(glob.glob(root_dir + '*')):
        if keyword in name:
            if ignore_keyword:
                continue
            else:
                mask_list.append(io.imread(name).astype(dtype))
        elif ignore_keyword:
            mask_list.append(io.imread(name).astype(dtype))
        else:
            continue
    return mask_list

In [5]:
# file root
file_root_0 = path + 'data/2P_functional/testing/'

file_root_1 = path + 'data/2P_functional/CP/cp_500epochs/benchmarking/'
file_root_2 = path + 'data/2P_functional/CP/cp_1000epochs/benchmarking/'
file_root_3 = path + 'data/2P_functional/CP/cp_2000epochs/benchmarking/'
file_root_4 = path + 'data/2P_functional/CP/cp_5000epochs/benchmarking/'
file_root_5 = path + 'data/2P_functional/CP/cp_notrain_autodiam/benchmarking/'
file_root_6 = path + 'data/2P_functional/CP/cp_notrain_diam9/benchmarking/'
file_root_7 = path + 'data/2P_functional/PyTC/PyC/benchmarking/'
file_root_8 = path + 'data/2P_functional/SD/sd_500epochs/benchmarking/'
file_root_9 = path + 'data/2P_functional/SD/sd_1000epochs/benchmarking/'
file_root_10 = path + 'data/2P_functional/SD/sd_2000epochs/benchmarking/'
file_root_11 = path + 'data/2P_functional/old/GT/'

vol_list = load_2d_masks(file_root_0, keyword='mask', ignore_keyword=True)
image_stack = np.array(vol_list)

gt = load_2d_masks(file_root_0, keyword='masks')
old_gt = load_2d_masks(file_root_11, keyword='masks')

cp_500 = load_2d_masks(file_root_1, keyword='masks')
cp_1000 = load_2d_masks(file_root_2, keyword='masks')
cp_2000 = load_2d_masks(file_root_3, keyword='masks')
cp_5000 = load_2d_masks(file_root_4, keyword='masks')
cp_nt_autodiam = load_2d_masks(file_root_5, keyword='masks')
cp_nt_diam9 = load_2d_masks(file_root_6, keyword='masks')

pc = load_2d_masks(file_root_7, keyword='box')

sd_500 = load_2d_masks(file_root_8, keyword='box')
sd_1000 = load_2d_masks(file_root_9, keyword='box')
sd_2000 = load_2d_masks(file_root_10, keyword='box')

In [6]:
models = [pc, cp_500, cp_1000, cp_2000, cp_5000, cp_nt_autodiam, cp_nt_diam9, sd_500, sd_1000, sd_2000]
model_names = ['PyTC', 'Cellpose (500 Epochs)', 'Cellpose (1000 Epochs)', 'Cellpose (2000 Epochs)', 'Cellpose (5000 Epochs)', 'Cellpose (untrained & autodiam)', 'Cellpose (untrained & diam9)', 'Stardist (500 Epochs)', 'Stardist (1000 Epochs)', 'Stardist (2000 Epochs)']
benchmarkers = []

for i, (data, name) in enumerate(zip(models, model_names)):
    print(name)
    temp_gt = gt.copy()
    temp_pred = data.copy()
    for i, (gt_instance, pred_instance) in enumerate(zip(temp_gt, temp_pred)):
        if np.unique(pred_instance).shape[0] == 1:
            temp_pred.pop(i)
            temp_gt.pop(i)
    benchmarker = Eval(temp_gt, temp_pred, resolution=(0.6, 0.6), model_name=name, name_data='2P Functional Planes')
    benchmarker.accumulate()
    benchmarkers.append(benchmarker)
'''
models = [pc]
model_names = ['PyTC']
for i, (data, name) in enumerate(zip(models, model_names)):
    print(name)
    temp_gt = old_gt.copy()
    temp_pred = data.copy()
    for i, (gt_instance, pred_instance) in enumerate(zip(temp_gt, temp_pred)):
        if np.unique(pred_instance).shape[0] == 1:
            temp_pred.pop(i)
            temp_gt.pop(i)
    benchmarker = Eval(temp_gt, temp_pred, resolution=(0.6, 0.6), model_name=name, name_data='2P Functional Planes')
    benchmarker.accumulate()
    benchmarkers.append(benchmarker)
'''

PyTC


/Users/Paul/anaconda3/envs/py38/lib/python3.7/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/Users/Paul/anaconda3/envs/py38/lib/python3.7/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Cellpose (500 Epochs)
Cellpose (1000 Epochs)
Cellpose (2000 Epochs)
Cellpose (5000 Epochs)
Cellpose (untrained & autodiam)
Cellpose (untrained & diam9)
Stardist (500 Epochs)
Stardist (1000 Epochs)
Stardist (2000 Epochs)


"\nmodels = [pc]\nmodel_names = ['PyTC']\nfor i, (data, name) in enumerate(zip(models, model_names)):\n    print(name)\n    temp_gt = old_gt.copy()\n    temp_pred = data.copy()\n    for i, (gt_instance, pred_instance) in enumerate(zip(temp_gt, temp_pred)):\n        if np.unique(pred_instance).shape[0] == 1:\n            temp_pred.pop(i)\n            temp_gt.pop(i)\n    benchmarker = Eval(temp_gt, temp_pred, resolution=(0.6, 0.6), model_name=name, name_data='2P Functional Planes')\n    benchmarker.accumulate()\n    benchmarkers.append(benchmarker)\n"

In [7]:
benchmark = Benchmarker(benchmarkers)

In [8]:
benchmark_cp = Benchmarker(benchmarkers[1:7])
benchmark_sd = Benchmarker(benchmarkers[7:10])
benchmark_pc = Benchmarker([benchmarkers[0]])

In [9]:
import seaborn as sns
from matplotlib.gridspec import GridSpec
sns.set()

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 7), sharey=True)
ax1.set_ylim([0.0, 1.0])
ax2.set_ylim([0.0, 1.0])
ax3.set_ylim([0.0, 1.0])
ax1 = benchmark_cp.plot_error_bars(ax1, metric='mAP @ 0.5:0.95', title='Cellpose')
ax2 = benchmark_sd.plot_error_bars(ax2, metric='mAP @ 0.5:0.95', title='Stardist')
ax3 = benchmark_pc.plot_error_bars(ax3, metric='mAP @ 0.5:0.95', title='PyTC')

fig.suptitle("Mean Average Precision Scores")
plt.tight_layout()
plt.savefig('mAP_Boxplots.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
sns.set()
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
benchmark.plot_ap_curves(ax)
ax.set_title('Average Precision')
plt.savefig('AP_Curves.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
sns.set()
fig, axes = plt.subplots(1, 3, figsize=(14, 7), sharey=True)
fig.suptitle('Average Distance between Centroids (ADC)', fontsize=18)
axes[0] = benchmark_cp.plot_error_bars(axes[0], metric='adc', title='ADC')
axes[1] = benchmark_sd.plot_error_bars(axes[1], metric='adc', title='ADC')
axes[2] = benchmark_pc.plot_error_bars(axes[2], metric='adc', title='ADC')
#ax[1] = benchmark_pc.plot_error_bars(ax[1], metric='adgc', title='PyTC')
#ax[2] = benchmark_sd.plot_error_bars(ax[2], metric='adgc', title='Stardist')
#sns.axlabel(xlabel="Models", ylabel="ADC [um]", fontsize=16, ax=ax)
axes[0].set_ylabel('Euclidean Distance [µm]', fontsize=14)
plt.tight_layout()
plt.savefig('ADC_Boxplots.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
benchmarkers[2].name = 'Cellpose'
benchmarkers[6].name = 'Cellpose (pretrained)'
benchmarkers[7].name = 'Stardist'
best = [benchmarkers[2]] + benchmarkers[6:8] + [benchmarkers[0]]
best_benchmark = Benchmarker(best)

In [13]:
sns.set()
fig = plt.figure(constrained_layout=True, figsize=(20, 8.5))
fig.suptitle('Two-Photon Functional Planes - Neuronal Cell Body Segmentation Results', fontsize=24)
font_size=12
gs = GridSpec(2, 5, figure=fig)
ax1 = fig.add_subplot(gs[:, :2])
ax2 = fig.add_subplot(gs[0, 2])
ax3 = fig.add_subplot(gs[0, 3])
ax4 = fig.add_subplot(gs[0, 4])
ax5 = fig.add_subplot(gs[1, 2])
ax6 = fig.add_subplot(gs[1, 3])
ax7 = fig.add_subplot(gs[1, 4])

best_benchmark.plot_ap_curves(ax1, fontsize=12)
ax1.set_title('Average Precision across IoU thresholds')

ax2 = best_benchmark.plot_error_bars(ax2, metric='mAP @ 0.5:0.95', title='mean Average Precision (mAP) @ 0.5:0.95', fontsize=font_size)
ax2.set_ylim([0.0, 1.0])
ax3 = best_benchmark.plot_error_bars(ax3, metric='AP @ 0.5', title='Average Precision (AP) @ 0.5', fontsize=font_size, set_legend=False)
ax4 = best_benchmark.plot_error_bars(ax4, metric='AP @ 0.75', title='Average Precision (AP) @ 0.75', fontsize=font_size, set_legend=False)

ax5 = best_benchmark.plot_error_bars(ax5, metric='adc', title='ADC', fontsize=font_size)
ax6 = best_benchmark.plot_error_bars(ax6, metric='adpc', title='ADPC (FN Error)', fontsize=font_size, set_legend=False)
ax7 = best_benchmark.plot_error_bars(ax7, metric='adgc', title='ADGC (FP Error)', fontsize=font_size, set_legend=False)
ax5.set_ylim([0.0, 8.0])
ax6.set_ylim([0.0, 8.0])
ax7.set_ylim([0.0, 8.0])

ax5.set_ylabel('Euclidean Distance [µm]', fontsize=font_size)
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12

plt.tight_layout()

plt.savefig(path+'figures/2P_Functional.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/Paul/anaconda3/envs/py38/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 


In [16]:
import random
from CellAnalysis.utils import label2rgb_3D
rows = 6
columns = 8
fig, axes = plt.subplots(rows, columns, figsize=(columns*3, rows*3))
#indices = np.random.randint(0, image_stack.shape[0], rows*columns)
#indices = [index for index in random.sample(range(image_stack.shape[0]), rows*columns)]
axes[0, 0].set_title('PyC Prediction')
axes[0, 1].set_title('GT Mask')
axes[0, 2].set_title('PyC Prediction')
axes[0, 3].set_title('GT Mask')
axes[0, 4].set_title('PyC Prediction')
axes[0, 5].set_title('GT Mask')
for i in range(rows*columns):
    if i % 2 == 0:
        #ind = indices[i]
        ind = int(i/2)
        image_gt = label2rgb_3D(pc[ind], image_stack[ind, :, :], alpha=0.3)
        image_pred = label2rgb_3D(gt[ind], image_stack[ind, :, :], alpha=0.3)
        
        axes[int(i/columns), i%columns].imshow(image_gt)
        axes[int(i/columns), i%columns].grid(False)
        axes[int(i/columns), i%columns].axes.get_xaxis().set_visible(False)
        axes[int(i/columns), i%columns].axes.get_yaxis().set_visible(False)
        i += 1
        axes[int(i/columns), i%columns].imshow(image_pred)
        axes[int(i/columns), i%columns].grid(False)
        axes[int(i/columns), i%columns].axes.get_xaxis().set_visible(False)
        axes[int(i/columns), i%columns].axes.get_yaxis().set_visible(False)
    else:
        continue
plt.tight_layout()
plt.savefig('2P_Planes_PyC_visualized.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
#plt.savefig('2P_Planes_Vol.png')

In [22]:
import matplotlib.animation as animation
from CellAnalysis.utils import gray2rgb




sns.set()
fig = plt.figure(figsize=(20, 8.5))
fig.suptitle('Two-Photon Functional Planes - Neuronal Cell Body Segmentation Results', fontsize=24)
font_size=12
gs = GridSpec(2, 5, figure=fig)
ax1 = fig.add_subplot(gs[:, :2])
ax2 = fig.add_subplot(gs[0, 2])
ax3 = fig.add_subplot(gs[0, 4:6])
ax4 = fig.add_subplot(gs[1, 4:6])
#ax4 = fig.add_subplot(gs[0, 4])
ax5 = fig.add_subplot(gs[0, 3])
ax6 = fig.add_subplot(gs[1, 2])
ax7 = fig.add_subplot(gs[1, 3])

best_benchmark.plot_ap_curves(ax1, fontsize=12)
ax1.set_title('Average Precision across IoU thresholds')

ax2 = best_benchmark.plot_error_bars(ax2, metric='mAP @ 0.5:0.95', title='mean Average Precision (mAP) @ 0.5:0.95', fontsize=font_size, set_legend=True, legend_loc='upper left')
#ax3 = benchmark_best.plot_error_bars(ax3, metric='AP @ 0.5', title='Average Precision (AP) @ 0.5', fontsize=font_size, set_legend=True)
#ax4 = benchmark_best.plot_error_bars(ax4, metric='AP @ 0.75', title='Average Precision (AP) @ 0.75', fontsize=font_size, set_legend=True, legend_loc='lower left')
ax2.set_ylim([0.0, 1.1])
#ax3.set_ylim([0.0, 1.2])
#ax4.set_ylim([0.0, 1.2])

ax5 = best_benchmark.plot_error_bars(ax5, metric='adc', title='ADC', fontsize=font_size, set_legend=False, legend_loc='upper left')
ax6 = best_benchmark.plot_error_bars(ax6, metric='adpc', title='ADPC (FN Error)', fontsize=font_size, set_legend=True, legend_loc='upper right')
ax7 = best_benchmark.plot_error_bars(ax7, metric='adgc', title='ADGC (FP Error)', fontsize=font_size, set_legend=True, legend_loc='upper left')
ax5.set_ylim([0.0, 10.0])
ax6.set_ylim([0.0, 10.0])
ax7.set_ylim([0.0, 10.0])

ax5.set_ylabel('Euclidean Distance [µm]', fontsize=font_size)
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12

ax3.set_xticks([])
ax3.set_yticks([])
#ax3.set_title('Cellpose Segmentation Results')
ax3.grid(False)

ax4.set_xticks([])
ax4.set_yticks([])
#ax3.set_title('Cellpose Segmentation Results')
ax4.grid(False)


vol_1 = gray2rgb(image_stack[2])
image_pred = label2rgb_3D(sd_500[2], image_stack[2, :, :], alpha=0.2)
image_gt = label2rgb_3D(gt[2], image_stack[2, :, :], alpha=0.2)
image_list_1 = [vol_1, image_gt, vol_1, image_pred]
#title_list = ['Raw Image', 'Ground Truth Mask', 'Raw Image', 'Stardist Prediction Mask']

vol_2 = gray2rgb(image_stack[23])
image_pred = label2rgb_3D(sd_500[23], image_stack[23, :, :], alpha=0.2)
image_gt = label2rgb_3D(gt[23], image_stack[23, :, :], alpha=0.2)
image_list_2 = [vol_2, image_gt, vol_2, image_pred]
title_list = ['Raw Image', 'Ground Truth Mask', 'Raw Image', 'Stardist Prediction Mask']


container = []
for i in range(30):
    for im1, im2, title in zip(image_list_1, image_list_2, title_list):
        #ax3.set_title(title, fontsize=font_size)
        image1 = ax3.imshow(im1, animated=True)
        text_title1 = ax3.text(0.5,1.05, '{}'.format(title), 
                        size=plt.rcParams["axes.titlesize"],
                        ha="center", transform=ax3.transAxes, )
        image2 = ax4.imshow(im2, animated=True)
        text_title2 = ax4.text(0.5,1.05, '{}'.format(title), 
                        size=plt.rcParams["axes.titlesize"],
                        ha="center", transform=ax4.transAxes, )
        container.append([image1, text_title1, image2, text_title2])


ani = animation.ArtistAnimation(fig, container, interval=2000, repeat=True, repeat_delay=0, blit=True)
plt.tight_layout()

#plt.savefig('EM.png')


# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame

Writer = animation.writers['ffmpeg']
writer = Writer(fps=0.2, metadata=dict(artist='Me'), bitrate=1800)
#ani.save('2P_animation.gif', writer='imagemagick', fps=0.5)
ani.save('2P_animation.mp4', writer=writer)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
from CellAnalysis.utils import gray2rgb
vol_2 = image_stack[2]
image_gt = label2rgb_3D(sd_500[2], image_stack[2, :, :], alpha=0.2)
image_pred = label2rgb_3D(gt[2], image_stack[2, :, :], alpha=0.2)

io.imsave('volume.tif', gray2rgb(vol_2))
io.imsave('gt.tif', image_gt)
io.imsave('pred.tif', image_pred)